## Synthetic dataset generation
Author: Lin Lee Cheong <br>
Date: 12/12/ 2020 <br>

Goal of this synthetic dataset is to create datasets to help understand how different relationships between tokens affect attention, SHAP and other interpretability factors.
- length of events (30, 300, 900)
- spacing between 2+ coupled events, i.e. order of sequence matters
- amount of noise, i.e. performance vs interpretability
- vocabulary space

In [1]:
import yaml
import string
import os
import pandas as pd

from utils import *

In [2]:
%load_ext autoreload

%autoreload 2

In [15]:
TOKEN_NAMES_FP = './tokens.yaml'

SEQ_LEN = 900 #300 #900

TRAIN_FP = 'data/train_seq{}.csv'.format(SEQ_LEN)
VAL_FP = 'data/val_seq{}.csv'.format(SEQ_LEN)
TEST_FP = 'data/test_seq{}.csv'.format(SEQ_LEN)

UID_COLNAME = 'patient_id'

TRAIN_NROWS = 3000
VAL_NROWS = 1000
TEST_NROWS = 1000

UID_LEN = 10


In [16]:
#Load tokens from yaml file path
tokens = load_tokens(TOKEN_NAMES_FP)
for key in tokens.keys():
    print(f"{key}: {len(tokens[key])} tokens")

adverse_tokens: 4 tokens
adverse_helper_tokens: 6 tokens
adverse_unhelper_tokens: 5 tokens
noise_tokens: 15 tokens


### Simple dataset

Get simple dataset:
- positive set: (+++, 1 major + a helper), (++, 1 major), (+, 3 helper)
- negative set: (---, 3 unhelper), (--, 1 helper + 2 unhelper), (-, 2 helper + 1 unhelper)


**NOTES**<br>
n_ppp_adverse = 2000 # 1 adverse event + 1 helper event <br>
n_pp_adverse = 2000 # 1 adverse event <br>
n_p_adverse = 2000 # 3 helper events <br><br>
n_nnn_adverse = 2000 # 3 unhelper events <br>
n_nn_adverse = 2000 # 1 helper + 2 unhelper <br>
n_n_adverse = 2000 # 2 helper + 1 unhelper <br>

In [17]:
train_count_dict = {
    'n_ppp_adverse': TRAIN_NROWS,
    'n_pp_adverse': TRAIN_NROWS,
    'n_p_adverse': TRAIN_NROWS,
    'n_nnn_adverse': TRAIN_NROWS,
    'n_nn_adverse': TRAIN_NROWS,
    'n_n_adverse': TRAIN_NROWS
}

val_count_dict = {
    'n_ppp_adverse': VAL_NROWS,
    'n_pp_adverse': VAL_NROWS,
    'n_p_adverse': VAL_NROWS,
    'n_nnn_adverse': VAL_NROWS,
    'n_nn_adverse': VAL_NROWS,
    'n_n_adverse': VAL_NROWS
}

test_count_dict = {
    'n_ppp_adverse': TEST_NROWS,
    'n_pp_adverse': TEST_NROWS,
    'n_p_adverse': TEST_NROWS,
    'n_nnn_adverse': TEST_NROWS,
    'n_nn_adverse': TEST_NROWS,
    'n_n_adverse': TEST_NROWS
}

In [18]:
train_simple_data = get_simple_dataset(
    seq_len=SEQ_LEN, uid_len=UID_LEN, uid_colname=UID_COLNAME, count_dict=train_count_dict, tokens=tokens)

val_simple_data = get_simple_dataset(
    seq_len=SEQ_LEN, uid_len=UID_LEN, uid_colname=UID_COLNAME, count_dict=val_count_dict, tokens=tokens)

test_simple_data = get_simple_dataset(
    seq_len=SEQ_LEN, uid_len=UID_LEN, uid_colname=UID_COLNAME, count_dict=test_count_dict, tokens=tokens)

dataset: (18000, 903)
ratio:
0    0.500167
1    0.499833
Name: label, dtype: float64

dataset: (6000, 903)
ratio:
0    0.501
1    0.499
Name: label, dtype: float64

dataset: (6000, 903)
ratio:
1    0.508833
0    0.491167
Name: label, dtype: float64



In [19]:
save_csv(train_simple_data, TRAIN_FP)
save_csv(val_simple_data, VAL_FP)
save_csv(test_simple_data, TEST_FP)

In [20]:
df = pd.read_csv(TRAIN_FP)
print(df.shape)
df.head()

(18000, 903)


,index,899,898,897,896,895,894,893,892,891,...,7,6,5,4,3,2,1,0,label,patient_id
0,58,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,hay_fever_N,backache_N,quad_injury_N,quad_injury_N,ACL_tear_N,pneumonia_H,pneumonia_H,cardiac_rehab_U,0,W522K3U1NM
1,583,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,myopia_N,dental_exam_N,peanut_allergy_N,quad_injury_N,ingrown_nail_N,myopia_N,hay_fever_N,tachycardia_H,0,4PY82TXWUI
2,2882,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,eye_exam_N,ACL_tear_N,annual_physical_N,ACL_tear_N,myopia_N,headache_N,foot_pain_N,apnea_H,1,HD8EMMXS9G
3,2159,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,quad_injury_N,ingrown_nail_N,foot_pain_N,eye_exam_N,ACL_tear_N,peanut_allergy_N,hay_fever_N,myopia_N,1,A07ZWN6OWW
4,2317,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,<pad>,...,ingrown_nail_N,quad_injury_N,annual_physical_N,peanut_allergy_N,eye_exam_N,eye_exam_N,foot_pain_N,annual_physical_N,1,I37X5VLIV1
